In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

In [2]:
with open('copenhagen_districts.geojson') as json_data:
    cph_data = json.load(json_data)
    
neighborhoods_data = cph_data['features']

# define the dataframe columns
column_names = ['Neighbourhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [3]:
for data in neighborhoods_data:
    neighborhood_name = data['properties']['name']
    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_latlon = np.mean(neighborhood_latlon[0][0],axis=0)
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]

    neighbourhood_price = data['properties']['k_price']
    neighbourhood_pop = data['properties']['total_pop']

    neighborhoods = neighborhoods.append({'Neighbourhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon,
                                          'Population': neighbourhood_pop,
                                          'Sqm_Price': neighbourhood_price},
                                          ignore_index=True)

In [4]:
cph_data = neighborhoods
cph_data

,Neighbourhood,Latitude,Longitude,Population,Sqm_Price
0,Osterbro,55.711846,12.589657,58235.0,0.688110
1,Valby,55.658006,12.499285,47539.0,0.204380
2,Kongens Enghave,55.645375,12.540390,11084.0,0.654452
3,Amager Ost,55.671191,12.632028,63794.0,0.384185
4,Bronshoj-Husum,55.710054,12.486824,44404.0,0.000000
5,Norrebro,55.696433,12.548778,48240.0,0.443041
6,Vesterbro,55.664996,12.552483,25046.0,0.764450
7,Indre By,55.685952,12.587683,55766.0,0.859703
8,Bispebjerg,55.716863,12.536427,46496.0,0.234804
9,Christianshavn,55.685673,12.609458,11760.0,1.000000


### Display of Neighbourhoods

In [5]:
address = 'Copenhagen, CPH'

geolocator = Nominatim(user_agent="cph_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Copenhagen are {}, {}.'.format(latitude, longitude))

# create map of Copenhagen using latitude and longitude values
map_cph = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cph)  
    
map_cph

The geograpical coordinate of Copenhagen are 55.6445495, 12.5451878.


## Define Foursquare Credentials and Version


In [6]:
CLIENT_ID = 'R01LINGO2WC45KLRLKT3ZHU2QENAO2IPRK2N2ELOHRNK4P3K' # your Foursquare ID
CLIENT_SECRET = '4JT1TWRMXMPLX5IOKNBAFU3L3ARXK4D5JJDPFK1CLRZM2ZVW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# defining radius and limit of venues to get
radius=2000
LIMIT=100

In [7]:
#import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
cph_venues = getNearbyVenues(names=cph_data['Neighbourhood'],
                                   latitudes=cph_data['Latitude'],
                                   longitudes=cph_data['Longitude']
                                  )

Osterbro
Valby
Kongens Enghave
Amager Ost
Bronshoj-Husum
Norrebro
Vesterbro
Indre By
Bispebjerg
Christianshavn
Fredriksberg
Amager Vest
Vanlose
